<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [6]:
# Import libraries here.

import pandas as pd 
import numpy as np 
import os

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [7]:
# Create an output folder.
os.mkdir("output")

# Create a files variable that contains all of our data files.
files = os.listdir("data")

In [8]:
files[:5]

['Jan 23.xlsx', 'Jan 19.xlsx', 'Jan 15.xlsx', 'Jan 7.xlsx', 'Jan 6.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [9]:
# Read in data from your city from January 1st. 

#reading in the first sheet 

df = pd.read_excel('data/Jan 1.xlsx')

In [10]:
#reading in other cities (for practice)

xls = pd.ExcelFile('data/Jan 1.xlsx')

# Now you can list all sheets in the file
xls.sheet_names


Denver = pd.read_excel(xls, sheet_name="Denver")

In [11]:
df = pd.read_excel('data/Jan 1.xlsx')

### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [12]:
df['price_usd'] = df['price_eu'] * 1.1

In [13]:
df.head()

,prodcode,price_eu,weight_kg,quantity,price_usd
0,4159,2.048141,8.812961,138,2.252955
1,4062,2.728485,9.331372,184,3.001334
2,4404,2.182498,8.937798,491,2.400748
3,4412,2.342694,7.067682,218,2.576964
4,4131,1.802839,9.551503,391,1.983123


In [14]:
df['weight_lb'] = df['weight_kg'] * 2.2

In [15]:
df.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515
1,4062,2.728485,9.331372,184,3.001334,20.529018
2,4404,2.182498,8.937798,491,2.400748,19.663156
3,4412,2.342694,7.067682,218,2.576964,15.548899
4,4131,1.802839,9.551503,391,1.983123,21.013306


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [16]:
plu = pd.read_csv("plu-codes.csv")

In [17]:
plu.head()

,product,plu_code
0,Apple (Fuji),4131
1,Apple (Gala),4134
2,Apricot,3302
3,Avocado,4225
4,Banana,4011


In [18]:
plu.rename(columns = {'plu_code': 'prodcode'}, inplace = True)

In [19]:
df = pd.merge(left=plu, right=df, left_on='prodcode', right_on='prodcode')

In [20]:
df['product'].unique()

array(['Apple (Fuji)', 'Apple (Gala)', 'Apricot', 'Avocado', 'Banana',
       'Blueberries', 'Strawberries', 'Corn', 'Cucumber', 'Eggplant',
       'Grapes', 'Kiwi', 'Lemon', 'Lychee', 'Mushroom', 'Onion', 'Orange',
       'Peach', 'Pear', 'Tomato'], dtype=object)

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [21]:
df.head()

,product,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,Apple (Fuji),4131,1.802839,9.551503,391,1.983123,21.013306
1,Apple (Fuji),4131,1.097750,7.616659,258,1.207525,16.756649
2,Apple (Fuji),4131,1.436785,1.647137,358,1.580463,3.623701
3,Apple (Fuji),4131,1.954375,5.072126,295,2.149813,11.158676
4,Apple (Fuji),4131,2.003414,3.576568,490,2.203756,7.868450


In [22]:
df = df.drop(['price_eu', 'weight_kg'], axis=1)

df.head()

,product,prodcode,quantity,price_usd,weight_lb
0,Apple (Fuji),4131,391,1.983123,21.013306
1,Apple (Fuji),4131,258,1.207525,16.756649
2,Apple (Fuji),4131,358,1.580463,3.623701
3,Apple (Fuji),4131,295,2.149813,11.158676
4,Apple (Fuji),4131,490,2.203756,7.868450


### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [23]:
df['date'] = 'Jan 1'

df['date'] = pd.to_datetime(df['date'], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 128
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   product    129 non-null    object        
 1   prodcode   129 non-null    int64         
 2   quantity   129 non-null    int64         
 3   price_usd  129 non-null    float64       
 4   weight_lb  129 non-null    float64       
 5   date       0 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 7.1+ KB


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [24]:
def process_data(file, city):
    data_ = pd.ExcelFile('data/' + file)
    data = pd.read_excel(xls, sheet_name = city)
    data['price_usd'] = data['price_eu'] * 1.1
    data['weight_lb'] = data['weight_kg'] * 2.2
    data = data.drop(['price_eu', 'weight_kg'], axis=1)
    data['date'] = file
    data['date'] = data['date'].str.replace(r'.xlsx', '')
    return data



Test your function out on a new file and city!

In [25]:
Boston = process_data('Jan 7.xlsx', 'Boston')

Boston

,prodcode,quantity,price_usd,weight_lb,date
0,4134,498,2.153334,17.287309,Jan 7
1,4012,384,1.003821,12.642420,Jan 7
2,4323,431,0.814828,19.247589,Jan 7
3,4078,457,1.728686,15.536389,Jan 7
4,4323,316,1.528344,2.458212,Jan 7
...,...,...,...,...,...
172,4131,309,0.864199,7.000650,Jan 7
173,4958,144,1.946128,3.702045,Jan 7
174,4272,455,2.057089,9.947643,Jan 7
175,3302,482,0.936777,2.459543,Jan 7


In [26]:
Denver = process_data('Jan 7.xlsx', 'Denver')

Denver

,prodcode,quantity,price_usd,weight_lb,date
0,3302,211,2.812466,11.757661,Jan 7
1,4309,245,1.244365,11.403375,Jan 7
2,4012,467,1.983985,11.826798,Jan 7
3,4011,132,1.449228,16.804181,Jan 7
4,4030,305,0.853514,21.821529,Jan 7
...,...,...,...,...,...
192,4272,107,1.068869,13.697700,Jan 7
193,4011,359,0.940999,21.094285,Jan 7
194,4650,349,2.128807,2.912771,Jan 7
195,4272,233,0.685163,12.302672,Jan 7


In [27]:
Boston

,prodcode,quantity,price_usd,weight_lb,date
0,4134,498,2.153334,17.287309,Jan 7
1,4012,384,1.003821,12.642420,Jan 7
2,4323,431,0.814828,19.247589,Jan 7
3,4078,457,1.728686,15.536389,Jan 7
4,4323,316,1.528344,2.458212,Jan 7
...,...,...,...,...,...
172,4131,309,0.864199,7.000650,Jan 7
173,4958,144,1.946128,3.702045,Jan 7
174,4272,455,2.057089,9.947643,Jan 7
175,3302,482,0.936777,2.459543,Jan 7


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [28]:
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 '.DS_Store',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

In [29]:
#making a combined list for Denver 

denver_combined = []

for file in files:
    try:
        denver_combined.append([process_data(file, 'Denver')])
    except: 
        print(file)
    

.DS_Store


In [30]:
denver_combined

[[     prodcode  quantity  price_usd  weight_lb    date
  0        3302       211   2.812466  11.757661  Jan 23
  1        4309       245   1.244365  11.403375  Jan 23
  2        4012       467   1.983985  11.826798  Jan 23
  3        4011       132   1.449228  16.804181  Jan 23
  4        4030       305   0.853514  21.821529  Jan 23
  ..        ...       ...        ...        ...     ...
  192      4272       107   1.068869  13.697700  Jan 23
  193      4011       359   0.940999  21.094285  Jan 23
  194      4650       349   2.128807   2.912771  Jan 23
  195      4272       233   0.685163  12.302672  Jan 23
  196      4131       110   1.746271  19.239602  Jan 23
  
  [197 rows x 5 columns]],
 [     prodcode  quantity  price_usd  weight_lb    date
  0        3302       211   2.812466  11.757661  Jan 19
  1        4309       245   1.244365  11.403375  Jan 19
  2        4012       467   1.983985  11.826798  Jan 19
  3        4011       132   1.449228  16.804181  Jan 19
  4        4030   

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [31]:
from pandas import DataFrame


denver = pd.DataFrame(np.array(denver_combined).reshape(6107,5))
denver.columns = ['prodcode','quantity','price_usd', 'weight_lb', 'date']

print(denver)

denver.head()

     prodcode quantity price_usd weight_lb    date
0        3302      211   2.81247   11.7577  Jan 23
1        4309      245   1.24437   11.4034  Jan 23
2        4012      467   1.98398   11.8268  Jan 23
3        4011      132   1.44923   16.8042  Jan 23
4        4030      305  0.853514   21.8215  Jan 23
...       ...      ...       ...       ...     ...
6102     4272      107   1.06887   13.6977   Jan 8
6103     4011      359  0.940999   21.0943   Jan 8
6104     4650      349   2.12881   2.91277   Jan 8
6105     4272      233  0.685163   12.3027   Jan 8
6106     4131      110   1.74627   19.2396   Jan 8

[6107 rows x 5 columns]


,prodcode,quantity,price_usd,weight_lb,date
0,3302,211,2.81247,11.7577,Jan 23
1,4309,245,1.24437,11.4034,Jan 23
2,4012,467,1.98398,11.8268,Jan 23
3,4011,132,1.44923,16.8042,Jan 23
4,4030,305,0.853514,21.8215,Jan 23


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [32]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}


In [44]:
files.remove('.DS_Store')

files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

In [49]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

for city, file_out in city_dict.items():
    data_list = [process_data(file, city) for file in files]
    data_full = pd.concat(data_list, axis=0)
    data_full["city"] = city
    data_full = data_full.loc[:, 
        ["city", "date", "prodcode", "quantity", "weight_lb", "price_usd"]
                             ]
    data_full.to_csv("output/" + file_out, index=False)

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [50]:
files_out = os.listdir("output")
results_list = [pd.read_csv("output/" + file) for file in files_out]
results = pd.concat(results_list, axis=0)

In [51]:
results.groupby("city")["quantity", "weight_lb", "price_usd"].mean()

<ipython-input-51-da01c770a73f>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  results.groupby("city")["quantity", "weight_lb", "price_usd"].mean()


,quantity,weight_lb,price_usd
city,,,
Atlanta,300.342246,11.836218,1.963143
Austin,300.323232,11.741192,1.835582
Boston,301.378531,12.054168,1.847483
Chicago,310.244604,11.858744,1.997456
Denver,295.563452,12.764020,1.902594
Los Angeles,305.020408,12.018517,1.884460
New York,287.875969,11.958079,1.944140
San Francisco,306.731959,12.047768,1.900404
Seattle,311.370000,11.714645,1.798299


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).